In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import forecast_tools as ft

import tensorflow.random.set_seed as tf_set_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Dropout
from tensorflow.keras.callbacks import  EarlyStopping, ModelCheckpoint

import emd

pd.options.plotting.backend = "plotly"
pd.set_option('precision', 2)

2023-01-16 23:47:18.610321: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-16 23:47:18.617434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 23:47:18.617457: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'tensorflow.random.set_seed'

# funcs

In [38]:
def get_dat(site,filename=None,features='all',emd=True,rename=False,start=None,end=None):

  if site == 'deere':
    df = pd.read_csv(  '/content/drive/MyDrive/Data/deere_load.csv',   
                          comment='#',
                          parse_dates=['Datetime (UTC-6)'],
                          index_col=['Datetime (UTC-6)'] )

  elif site == 'deere-supercleaned':
    df = pd.read_csv(  '/content/drive/MyDrive/Data/deere_load_supercleaned.csv',   
                          comment='#',
                          parse_dates=['Datetime (UTC-6)'],
                          index_col=['Datetime (UTC-6)'] )

  elif site == 'hyatt':
    df = pd.read_csv(  '/content/drive/MyDrive/Data/hyatt_load_IMFs.csv',   
                          comment='#',
                          parse_dates=['Datetime (UTC-10)'],
                          index_col=['Datetime (UTC-10)'] )       
    
  elif site == 'lajolla':
    df = pd.read_csv(  '/content/drive/MyDrive/Data/lajolla_load_IMFs.csv',   
                          comment='#',
                          parse_dates=['Datetime (UTC-8)'],
                          index_col=['Datetime (UTC-8)'] )  

  # elif site == 'nwe':
  #   df = pd.read_csv(   '/content/drive/MyDrive/Data/NWE/ca_actual.csv',   
  #                       comment='#',                 
  #                       parse_dates=['Date'],
  #                       index_col=['Date'])
  #   df = convert_nwe_data_to_vector(df)

  elif site == 'terna':
    file_path = '/content/drive/MyDrive/Data/terna_load_kw.csv'
    dfm = pd.read_csv(  file_path,
                        comment='#',
                        index_col=0)

    idx = pd.date_range(  start   = '2006-1-1 0:00',
                          end     = '2015-12-31 23:00',
                          freq    = 'H')

    df = pd.DataFrame(index=idx, data=np.empty(len(idx)), columns=['Load (kW)'])

    begin, end = 0, 24
    for i in range(dfm.shape[0]):
      dat = dfm.iloc[i].values
      df['Load (kW)'].iloc[begin:end] = dat
      begin, end = begin+24, end+24

    # 2006-03-26 02:00:00   NaN
    # 2007-03-25 02:00:00   NaN
    # 2008-03-30 02:00:00   NaN
    # 2009-03-29 02:00:00   NaN
    # 2010-03-28 02:00:00   NaN
    # 2011-03-27 02:00:00   NaN
    # 2012-03-25 02:00:00   NaN
    # 2013-03-31 02:00:00   NaN
    # 2014-03-30 02:00:00   NaN
    # 2015-03-29 02:00:00   NaN
    df = df.fillna(method='ffill')
    df['Load (kW)'][df['Load (kW)'].isna()]
    
  else:
    df = pd.read_csv(filename,
                     comment='#',
                     index_col=0,
                     parse_dates=True)
    
  if rename:
    df = df.rename(columns={df.columns[0]:'Load (kW)'})

  if df.columns[0] != 'Load (kW)':
    input('/// Warning pass rename=True to rename (enter to ack): ')
    #df = df.rename(columns={df.columns[0]:'Load (kW)'})
    
  if emd:
    df = emd_sift(df)
    
  if start and end:
    df = df.loc[start:end,:]
                                    
  # df['Day'] = df.index.dayofyear
  # df['Hour'] = df.index.hour
  # df['Weekday'] = df.index.dayofweek
  
  df['Day'] = np.abs(df.index.dayofyear - 182)
  df['Hour'] = np.abs(np.abs(df.index.hour-12)-12)
  df['Weekday'] = np.abs(df.index.dayofweek - 3)
  
  dppd = {'H':24,'15T':96,'T':1440}[df.index.inferred_freq]
    
  d = df['Load (kW)'].values.flatten()
  rmse_np1d = rmse(d[(dppd*1):],d[:-(dppd*1)])
  rmse_np7d = rmse(d[(dppd*7):],d[:-(dppd*7)])

  if rmse_np1d < rmse_np7d:
    np_days = 1
  else:
    np_days = 7
  
  if features=='all':
    return df, dppd, np_days
  else:
    return df[features], dppd, np_days
  
def one_hot_of_peaks(ds,freq='D'):
    df = pd.DataFrame(ds)
    df['peak'] = 0
    df.loc[df.groupby(pd.Grouper(freq=freq)).idxmax().iloc[:,0], 'peak'] = 1  
    return df['peak']     

def accuracy_one_hot(true,pred):
    """ Measure the accuracy of two one hot vectors, inputs can be 1d numpy or dataseries"""
    n_misses = sum(true != pred)/2     # every miss gives two 'False' entries
    return 1 - n_misses/sum(true)   # basis is the number of one-hots


def batch_generator(batch_size, sequence_length, num_x_signals, num_y_signals,
                    num_train, x_train_scaled, y_train_scaled):
    """
    Generator function for creating random batches of training-data.
    """    
    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)     
        
def rmse(y_true, y_pred):
  return np.sqrt(np.mean(np.square(y_true - y_pred)))

def emd_sift(df):
  imf = emd.sift.sift(df['Load (kW)'].values)

  for i in range(imf.shape[1]):
      df['IMF%s'%(i+1)] = imf[:,i]  

  return df           

# gru peak

In [44]:
for units in [6,12,24,48,72,96,128]:
    for sequence_length in [24,48,72,168]:
        for layers in [1,2,3]:
            for afuncs in [{'gru':'relu','dense':'relu'},{'gru':'relu','dense':'relu'}]:
            
                # data
                site = 'prpa'
                filename = 'data/PRPA_load_cleaned_mjw.csv'
                data_range = ['2021-6-7','2022-8-10']
                dir = 'models'
                features = ['Load (kW)',
                            'Day',
                            'Weekday',
                            'Hour',
                            'IMF1',                                
                            'IMF2',                                
                            'IMF3',
                            'IMF4',
                            'IMF5',
                            'IMF6',
                            'IMF7',
                            'IMF8',]
                targets = ['TargetsOH']

                # model
                rnn_type='gru'
                #units=24
                #layers=2
                #dropout=0
                #afuncs={'lstm':'relu','gru':'relu','dense':'relu'}

                # training
                #sequence_length=24
                epochs=100
                patience=20
                train_split = 0.9
                shift_steps = 1
                loss='binary_crossentropy'
                batch_size=1
                np.random.seed(42)
                tf_set_seed(42)

                # output
                verbose=0
                output = True
                plots = False
                metrics=['accuracy']

                # model filename
                t = datetime.now()
                path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_' + \
                                f'{t.hour:02}-{t.minute:02}-{t.second:02}_{rnn_type}-u{units}-l{layers}.keras'


                ### begin

                df,dppd,np_days = get_dat(site,filename,emd=True,rename=True,start=data_range[0],end=data_range[1])                    

                df['LoadOH'] =      one_hot_of_peaks(df[['Load (kW)']])
                df['TargetsOH'] =   one_hot_of_peaks(df[['Load (kW)']]).shift(-shift_steps)
                df['NP1dOH'] =      one_hot_of_peaks(df[['Load (kW)']]).shift(np_days*dppd)
                df = df.dropna()

                # split
                num_data = len(df)
                num_train = int(train_split * num_data)
                df_train = df.iloc[:num_train,:]
                df_valid = df.iloc[num_train:,:]

                feature_scaler = MinMaxScaler()
                X_train = feature_scaler.fit_transform(df_train[features].values)
                X_valid = feature_scaler.fit_transform(df_valid[features].values)

                y_train = df_train.TargetsOH.values[:,np.newaxis]
                y_valid = df_valid.TargetsOH.values[:,np.newaxis]

                generator =    batch_generator( batch_size,
                                                sequence_length,
                                                num_x_signals=len(features),
                                                num_y_signals=len(targets),
                                                num_train=num_train,
                                                x_train_scaled=X_train,
                                                y_train_scaled=y_train)   

                #X_batch, y_batch = next(generator)

                X_valid = X_valid[np.newaxis,:,:]
                y_valid = y_valid[np.newaxis,:,:] 

                if rnn_type == 'gru':
                    model = Sequential()
                    model.add( GRU( units=units,
                                    return_sequences=True,
                                    input_shape=(None, len(features),),
                                    activation=afuncs[rnn_type]) )
                    if layers > 1:
                        model.add( GRU( units=units,
                                        return_sequences=True,
                                        activation=afuncs[rnn_type]) )
                    if layers > 2:
                        model.add( GRU( units=units,
                                        return_sequences=True,
                                        activation=afuncs[rnn_type]) )        
                    model.add( Dense(len(targets), activation=afuncs['dense']) )

                elif rnn_type == 'lstm':
                    if layers==1:
                        model = Sequential([LSTM(units=units, return_sequences=True,
                                                input_shape=(None,len(features)),activation=afuncs[rnn_type]),
                                            Dense(units=len(targets), activation=afuncs[rnn_type]) ] )
                    elif layers==2:
                        model = Sequential([LSTM(units=units, return_sequences=True,
                                                input_shape=(None,len(features)),activation=afuncs[rnn_type]),
                                            LSTM(units=units, return_sequences=True, activation=afuncs[rnn_type]), 
                                            Dense(units=len(targets), activation=afuncs[rnn_type]) ] )

                model.compile( loss=loss, optimizer='adam', metrics=metrics )
                model.summary()                

                hx = model.fit( x=generator,
                                epochs=epochs,
                                steps_per_epoch=100,
                                validation_data=(X_valid,y_valid),
                                callbacks=[ ModelCheckpoint(filepath=path_checkpoint,
                                                            monitor='val_loss',
                                                            verbose=verbose,
                                                            save_weights_only=True,
                                                            save_best_only=True         ),
                                            EarlyStopping  (monitor='val_loss',
                                                            patience=patience,
                                                            verbose=verbose             )   ])        

                model.load_weights(path_checkpoint)
                print('\n/// Best Model Loss (valid)\n',model.evaluate(X_valid,y_valid)[0])

                y_valid_pred = model.predict(X_valid)
                y_valid_flat      = y_valid[:,:,0].flatten()
                y_valid_pred_flat = y_valid_pred[:,:,0].flatten()

                df_valid.loc[:,'y'] = y_valid_flat
                df_valid.loc[:,'y_pred'] = y_valid_pred_flat    

                acc = accuracy_one_hot(df_valid['y'],one_hot_of_peaks(df_valid['y_pred']))
                print('\n/// Accuracy\n',acc)

                pd.DataFrame(hx.history).plot(title=f'Accuracy = {acc:.3f}')  

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_9 (GRU)                 (None, None, 24)          2736      
                                                                 
 gru_10 (GRU)                (None, None, 24)          3600      
                                                                 
 dense_4 (Dense)             (None, None, 1)           25        
                                                                 
Total params: 6,361
Trainable params: 6,361
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 3s 16ms/step - loss: 0.3143 - accuracy: 0.0417 - val_loss: 0.1970 - val_accuracy: 0.0417
Epoch 2/100
100/100 [==============================] - 1s 14ms/step - loss: 0.1848 - accuracy: 0.0425 - val_loss: 0.1861 - val_accuracy: 0.0417
Epoch 3/100
100/100 [=======

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
model.load_weights(path_checkpoint)
y_valid_pred = model.predict(X_valid)
df_valid['y_pred'] = y_valid_pred.flatten()
#df_valid['y_pred'] = one_hot_of_peaks(df_valid.y_pred)
#df_valid['y_pred_xform'] = df_valid.y_pred.diff()
df_valid[['y','y_pred','NP1dOH']].head(168).plot()


1/1 [==============================] - 0s 128ms/step


/tmp/ipykernel_240506/1170676387.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# sandbox

In [41]:
df,dppd,np_days = get_dat(site,filename,emd=True,rename=True,start=data_range[0],end=data_range[1])
df

,Load (kW),IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,Day,Hour,Weekday
Datetime,,,,,,,,,,,,
2021-06-07 00:00:00,311.51,17.51,-56.78,-49.88,-9.30,1.45,3.15,0.32,405.04,24,0,3
2021-06-07 01:00:00,292.06,-9.53,-51.17,-48.19,-9.21,1.68,3.11,0.31,405.07,24,1,3
2021-06-07 02:00:00,279.75,-30.10,-44.99,-46.41,-9.12,1.91,3.08,0.29,405.09,24,2,3
2021-06-07 03:00:00,273.09,-45.55,-38.38,-44.54,-9.02,2.15,3.05,0.27,405.11,24,3,3
2021-06-07 04:00:00,272.58,-55.24,-31.46,-42.59,-8.92,2.38,3.02,0.26,405.13,24,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10 19:00:00,634.63,128.99,12.11,17.50,1.15,17.07,-25.91,63.97,419.76,40,5,1
2022-08-10 20:00:00,599.99,94.29,11.28,18.44,1.17,16.99,-25.89,63.95,419.76,40,4,1
2022-08-10 21:00:00,555.08,49.33,10.45,19.38,1.18,16.91,-25.87,63.94,419.77,40,3,1
